In [ ]:
# 라이브러리 설치 
# !pip install konlpy

In [ ]:
from konlpy.tag import Okt
okt = Okt()
print(okt.morphs('나는 학교에 간다'))

# 데이터의 분할
- KFold의 분할 방식 
    - KFold
        - 무작위로 데이터를 폴드화 
    - StratifiedKFold
        - 계층화를 유지하면서 폴드화 

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold

data = {
    'document' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
    'label' : [1, 1, 0, 0, 1, 0, 0, 1], 
    'id' : ['a', 'a', 'b', 'b', 'c', 'c', 'd', 'd']
}
df = pd.DataFrame(data)
df

In [ ]:
# 일반적인 KFold 
X = df['document']
Y = df['label']
groups = df['id']

k_folds = KFold(n_splits=2, shuffle=True, random_state=42)
s_folds = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
sg_fold = StratifiedGroupKFold(n_splits=2, shuffle=True, random_state=42)

In [ ]:
for x_idx, y_idx in k_folds.split(X, Y):
    print(df.loc[x_idx])
    print(df.loc[y_idx])
    break

In [ ]:
# 계층화 KFold
for x_idx, y_idx in s_folds.split(X, Y):
    print(df.loc[x_idx])
    print(df.loc[y_idx])
    break

In [ ]:
# 계층별 그룹화 KFold
for x_idx, y_idx in sg_fold.split(X, Y, groups):
    print(df.loc[x_idx])
    print(df.loc[y_idx])
    break

In [ ]:
# 네이버 영화 리뷰 (rating_train.txt)파일 
# pandas를 이용하여 txt 파일을 로드 
# 파일을 로드하는데 데이터 간의 분리법은 tab으로 이루어져있다. 
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df

In [ ]:
# 정보를 확인 
df.info()

In [ ]:
# info()를 통해서 document 컬럼의 결측치가 확인 -> 5개
# 150000개에서 5개의 데이터는 제거 가능 
# case1 (isna() + any() -> 인덱스의 조건식으로 해당 조건을 부정하여 사용)
df.loc[~df.isna().any(axis=1)]

In [ ]:
df.dropna(inplace=True)

In [ ]:
# document가 같은 문장이라면 문장을 하나만 두고 나머지는 제거 
# 중복 데이터가 존재하는가?
df['document'].value_counts()

In [ ]:
# 중복된 데이터는 제거 
# 제거하기 전의 데이터의 개수 
before = len(df)
# drop_duplicates() : 데이터의 중복을 제거하는 함수
df = df.drop_duplicates(['document']).reset_index(drop=True)
after = len(df)
print("중복 데이터 제거한 행의 개수 : ", before - after)

In [ ]:
# id 컬럼의 유일한 데이터들의 길이를 확인 
print(len(
    df['id'].unique()
))

In [ ]:
len(df)

In [ ]:
# label 컬럼의 데이터의 개수를 확인 
df['label'].value_counts()

In [ ]:
# train, validation, test 데이터셋으로 8:1:1 정도의 비율로 데이터를 분할
# label의 비율에 맞게 데이터를 나눠준다. 
from sklearn.model_selection import train_test_split
# sklearn에는 3개의 데이터셋으로 나눠주는 함수를 존재x
# train_test_split를 2번 사용
X = df['document'].values
Y = df['label'].values
# test 데이터셋을 10%로 먼저 나눠준다. 
X_temp, X_test, Y_temp, Y_test = train_test_split(
    X, Y, test_size=0.1, random_state=42, stratify=Y
)
# validation 데이터셋을 11% 정도로 나눠준다. (label데이터의 비율에 맞게)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_temp, Y_temp, test_size=0.11, random_state=42, stratify=Y_temp
)

In [ ]:
# 데이터의 분할 정도를 확인
print(len(X_train) / len(X) * 100)
print(len(X_val) / len(X) * 100)
print(len(X_test) / len(X) * 100)

In [ ]:
print(pd.Series(Y_train).value_counts())

In [ ]:
print(pd.Series(Y_val).value_counts())

In [ ]:
# 계층 폴드화 -> 학습 데이터를 데이터를 분할/학습하여 일반적인 성능을 나타내는 방법
# 폴드화, 하이퍼 파라미터 탐색과 같이 사용
folds = []
# enumerate() -> 리스트에서 위치와 값으로 데이터를 나눠서 되돌려준다. 
# s_folds.split(X_train, Y_train) -> 결과 값이 ( (tr_idxs, va_idxs) )
for fold, (tr_idx, va_idx) in enumerate(
    s_folds.split(X_train, Y_train)):
    folds.append(
        {
            'fold' : fold, 
            'tr_idx' : tr_idx, 
            'va_idx' : va_idx
        }
    )
folds

In [ ]:
# fold에서 첫번째 데이터에서 tr_idx 가지고 Y_train의 0,1의 비율을 확인 
test_idx = folds[0]['tr_idx']
pd.Series(Y_train[test_idx]).value_counts()

# 단어의 토큰화 
- 문장을 단어로 잘라준다. 
    - 공백을 기준으로 문자를 자른다. 
        -영문에서는 사용 가능, 한글에서는 의미가 소실되는 경우가 발생 
    - 형태소를 사용하여 문자를 나눠준다. 
        - 국어 사전을 로드하여 단어별로 나눠준다. 

In [ ]:
# 공백을 기반으로 데이터를 나눈다. 
text = '나는 학교에 간다'

In [ ]:
tokens = text.split()
print(tokens)

In [ ]:
text2 = "Hello World"
tokens2 = text2.split()
print(tokens2)

In [ ]:
from konlpy.tag import Okt
okt = Okt()

print(okt.morphs(text)) # 단어별로 나눠주는 함수
text_pos = okt.pos(text)
print(okt.pos(text))    # 단어와 단어의 종류를 출력 

In [ ]:
text_pos.remove( ('는','Josa') )   # 하나씩 정리하는 방법 ( 매우 귀찬 )

In [ ]:
text_pos

In [ ]:
# 반복문을 이용하여 각원소들을 대입하여 실행 
for t in text_pos:
    # print(t)
    # t -> tuple -> 두번째 값이 'Josa'가 아니라면
    if t[1] != 'Josa':
        print(t[0])

In [ ]:
okt.nouns(text)

In [ ]:
# Okt 로드한 데이터를 이용하여 Okt 형태소 분석 
for idx, t in enumerate(X_train):
    if idx == 5:
        break
    _pos = okt.morphs(t)
    print(_pos)

In [ ]:
!pip install Korpora 

In [ ]:
# from Korpora import Korpora 
# data = Korpora.load('nsmc')

In [ ]:
# 형태소를 이용한 토큰화 
# !pip install sentencepiece

In [ ]:
# sentencepiece 모듈을 이용하여 형태소 분석 
# 모델 학습 
# train txt, test txt 파일을 모두 로드하여 학습에 대입 
df_tr = pd.read_csv("../data/ratings_train.txt", sep='\t').dropna()
df_te = pd.read_csv("../data/ratings_test.txt", sep='\t').dropna()

In [ ]:
# 두개의 데이터프레임을 단순 행결합(union 결합)
total_df = pd.concat( [df_tr['document'], df_te['document']], 
                     axis=0, ignore_index=True )

In [ ]:
total_df.info()

In [ ]:
# 모델에 학습 시키기 전에 파일로 미리 저장 
total_df.to_csv('test.txt', index=False, header=False)

In [ ]:
# 모델을 생성 
import sentencepiece as spm

spm.SentencePieceTrainer.Train(
    input = 'test.txt',     # 학습에서 사용할 텍스트 파일
    model_prefix = 'ko_unigram', # unigram -> 한글 적합한 형태 (한단어씩 잘라서 표현) -> 모델명
    vocab_size = 8000,  # 단어 사전의 크기(모델의 크기) -> 8000, 16000, 32000
    model_type = 'unigram',     # 토큰의 생성 방식 
                                # unigram -> BERT, KoGPT등에서 사용이 되는 언어 모델 방식
                                # bpe -> GPT-2 사용하는 방식 (한글에서는 부적합)
                                # char -> 문자 단위( 정보가 너무 짧게 쪼개져 있는 형태 )
                                # word -> 단어 단위(한국어에 부적합)
    character_coverage = 0.9995,    # 학습에 포한할 문자의 종류의 비율
                                    # 1.0 인 경우 모든 문자의 종류를 사용
                                    # 0.9995 -> 한글, 영문, 숫자 포함 
    input_sentence_size =  100000, # 학습 문장을 샘플링 
                                    # 모든 데이터를 사용하는게 제일 좋은 방법(시간이 오래 걸림)
                                    # 일부만 샘플링하여 사용하는 방법
    shuffle_input_sentence = True   # 샘플링시 문장의 순서를 섞어서 사용
                                    # 모델이 특정 순서에 편향되는것을 방지 
)

In [ ]:
# 생성된 모델을 이용하여 형태소 분석 
sp = spm.SentencePieceProcessor()
# 생성된 모델을 로드 
sp.load('ko_unigram.model')
text = '나는 학교에 간다'
print(sp.encode(text, out_type=str))

In [ ]:
# ▁ 특수 기호는 키보드 입력이 불가 
char = '\u2581'
print(char)

In [ ]:
for idx, t in enumerate(X_train):
    if idx == 5:
        break
    print(sp.encode(t, out_type=str))

In [ ]:
from konlpy.tag import Komoran

In [ ]:
komoran = Komoran()

In [ ]:
print(komoran.morphs(text))  # 형태소 나열
print(komoran.pos(text))    # (행태소, 동사) 튜플 나열
print(komoran.nouns(text))  # 명사만 출력

### komoran 동사를 일반적으로 사용하는 것들 
- 감성/의도 분석/리뷰 (가장 일반적)
    - NNG(일반명사), NNP(고유명사), VV(동사), VA(형용사), MAG(일반부사), SL(외국어)
- 명사 기반의 분류 (문서에 대한 분류 작업)
    - NNG(일반명사), NNP(고유명사), NR(수사), NP(대명사)
- 의미가 있는 단어를 최대한 포함하고 싶은 경우
    - NNG(일반명사), NNP(고유명사), VV(동사), VA(형용사), MAG(일반부사), MAJ(접속부사), IC(감탄사), SL(외국어)

In [ ]:
# 사용할 형태소의 종류
allow_pos = ['NNG', 'NNP', 'VV', 'VA']

# 선택한 형태소들을 추출하기 위한 함수를 정의 
def komoran_tokenize(text):
    # 선택한 형태소만 저장하는 빈 리스트를 생성 
    result = []
    for morph, pos in komoran.pos(text):
        if pos in allow_pos:
            result.append(morph)
    return result

for idx, t in enumerate(X_train):
    if idx == 5:
        break
    print(komoran_tokenize(t))

# 백터화 
- 토큰화 작업에서 단어들을 추출했다면 해당 단어들을 숫자형으로 변환
    - 숫자형태로 변환하는 이유는? -> 컴퓨터가 숫자로만 연산이 가능하기 때문에
- 숫자형태로 변환한 데이터를 학습 데이터로 이용, 정답은 lebel 데이터로 규칙을 생성해가는 과정 

In [ ]:
# one-hot encoding -> 하나의 리뷰에서 특정 단어가 포함되어 있는가?
df = pd.read_csv("../data/ratings_train.txt", sep='\t').dropna()
df.drop('id', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
# 전체의 텍스트를 이용해서 학습을 통한 단어를 습득한 뒤 
# 해당하는 단어들이 리뷰에 포함되어있는가?

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# 객체 생성
# 존재 여부만 파악 객체 생성
vectorizer = CountVectorizer(binary=True)

# 학습을 한 뒤 변환(data 대입) -> 데이터는 document에서 10개의 데이터
X = vectorizer.fit_transform(df['document'].head(5))
X

In [ ]:
# 학습한 단어들이 무엇인가 출력 (단어 사전)
vocab = vectorizer.get_feature_names_out()
print(vocab)

In [ ]:
# get_feature_names_out()의 단어를 포함하고 있는지 확인 
print(X.toarray())

In [ ]:
df['label'].head(5)

In [87]:
# Okt + CounterVectorizer 같이 사용 -> 토큰화 + 백터화

okt = Okt()

# 형태소 변환 함수 정의 
def okt_tokenize(text):
    # 특정 형태의 단어들만 추출한다. 
    # 명사, 동사, 형용사만 선택
    select_pos = ['Noun', 'Verb', 'Adjective']
    # (단어, 형태)를 출력하는 pos() 함수 이용
    # result = okt.morphs(text)
    result = [ 
        word for word, pos in okt.pos(text) if pos in select_pos
    ]
    # 위의 코드의 작동 방식
    # result2 = []
    # for word, pos in okt.pos(text):
    #     if pos in select_pos:
    #         result2.append(word)
    return result

# CounterVertorizer 생성 
vectorizer_okt = CountVectorizer(
    tokenizer= okt_tokenize, 
    lowercase= False, 
    binary=True
)

x_okt = vectorizer_okt.fit_transform(df['document'].head(5))

c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [88]:
print(vectorizer_okt.get_feature_names_out())

['가볍지' '교도소' '구먼' '늙어' '다그' '더빙' '던스트' '돋보였던' '래서' '목소리' '몬페' '무재' '밓었'
 '보고' '보는것을' '보였다' '보이기만' '솔직히' '스파이더맨' '않구나' '없다' '연기' '영화' '오버' '의' '이뻐'
 '이야기' '익살스런' '재미' '조정' '줄' '진짜' '짜증나네요' '초딩' '추천' '커스틴' '평점' '포스터' '했던'
 '흠']


In [ ]:
print(x_okt.toarray())

In [90]:
pd.DataFrame(
    x_okt.toarray(), 
    columns= vectorizer_okt.get_feature_names_out()
)

,가볍지,교도소,구먼,늙어,다그,더빙,던스트,돋보였던,래서,목소리,...,줄,진짜,짜증나네요,초딩,추천,커스틴,평점,포스터,했던,흠
0,0,0,0,0,0,1,0,0,0,1,...,0,1,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,1
2,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,1,0
